In [7]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import keras 

In [8]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
def check_dicom(filename): 
    print('Load file {} ...'.format(filename))
    ds = pydicom.dcmread(filename)
    #print(ds)
    
    # print image position, image type and body part
    #print('Image position is {}'.format(ds.ImagePositionPatient))   <== ERROR - THIS FIELD IS MISSING FROM THE DATASET
    #print('Patient position is {}'.format(ds.PatientPosition))
    #print('Image type is {}'.format(ds.Modality))
    #print('Body part is {}'.format(ds.BodyPartExamined))
    
    img = ds.pixel_array
    #print('Image shape is {}'.format(img.shape))
    #plt.imshow(img, cmap="gray")

    if ds.BodyPartExamined != 'CHEST' or ds.Modality != 'DX' or ds.PatientPosition not in ['PA', 'AP']:
        print('Invalid image file')
        img = None
    
    return img
    
from tensorflow.keras.applications.vgg16 import preprocess_input
from skimage.transform import resize

# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img, img_size): 
    img = resize(img, (img_size[1],img_size[2]), anti_aliasing=True)
    #print(img.shape)
    
    #preprocess the image
    #img = img_to_array(img)
    grey_image = preprocess_input(img)
    #print(grey_image.shape)
    
    proc_img = np.zeros((224,224,3))
    # to RGB (input of the first layer)
    proc_img[:, :, 0] = grey_image
    proc_img[:, :, 1] = grey_image
    proc_img[:, :, 2] = grey_image
    proc_img = proc_img.reshape(img_size)
    #print(proc_img.shape)
    
    return proc_img

from tensorflow.keras.models import model_from_json

# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):
    with open(model_path, "r") as json_file:
        loaded_model_json = json_file.read()
        my_model = model_from_json(loaded_model_json)

        my_model.load_weights(weight_path)
        return my_model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh): 
    pred_Y = model.predict(img, verbose = True)   
    
    if pred_Y > thresh:
        prediction = "Pneumonia"
    else:
        prediction = "No Pneumonia"
        
    return prediction 

In [9]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path = "my_model.json"
weight_path="{}_my_model.best.hdf5".format('xray_class')
my_model = load_model(model_path, weight_path)

IMG_SIZE=(1,224,224,3) # This might be different if you did not use vgg16
#img_mean = # we are using preprocess_input
#img_std = # we are using preprocess_input
    
thresh = 0.6315384  #loads the threshold for model classification 

# use the .dcm files to test your prediction
for i in test_dicoms:
    #img = np.array([])
    img = check_dicom(i)
    
    if img is None:
        continue
        
    img_proc = preprocess_image(img, IMG_SIZE)
    pred = predict_image(my_model,img_proc,thresh)
    print(pred)

Load file test1.dcm ...
1/1 [==============================] - 1s 1s/sample
No Pneumonia
Load file test2.dcm ...
1/1 [==============================] - 1s 1s/sample
No Pneumonia
Load file test3.dcm ...
1/1 [==============================] - 1s 1s/sample
No Pneumonia
Load file test4.dcm ...
Invalid image file
Load file test5.dcm ...
Invalid image file
Load file test6.dcm ...
Invalid image file
